<a href="https://colab.research.google.com/github/mjl-ai/TFM-Sentimiento-Politico-Transformers/blob/main/notebooks%20/TransformerLigeros_caso_balanceado_y_desbalanceado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#✅ TRANSFORMES LIGEROS ESCENARIO DATASET: BALANCEADO Y DESBALANCEADO



In [ ]:
# ============================================================
# TFM - NLP Sentimiento Político (Transformers ligeros)
# NOTEBOOK ÚNICO (COPIAR/PEGAR) - Ajustado a:
# ✅ EarlyStopping + load_best_model_at_end (como antes)
# ✅ Checkpoints SÍ se generan para entrenar, PERO:
#    - NO se incluyen en el ZIP
#    - se guardan fuera de la carpeta del ZIP y se BORRAN al final de cada run
# ✅ TOP 20 palabras GENERAL (EDA normalizado: text_eda) para:
#    - escenario desbalanceado
#    - escenario balanceado
# ✅ ZIP SOLO con TABLAS y FIGURAS (NO checkpoints)
# ✅ Indica cuántos registros se usan en entrenamiento (train split) por escenario
# ✅ + NUEVO: 3 gráficos de distribución de clases:
#    1) balanceado, 2) desbalanceado, 3) comparativo
# ✅ + NUEVO: tabla comparativa incluye métricas de LOSS
# ============================================================

!pip -q install -U "transformers==4.44.2" datasets evaluate accelerate scikit-learn wordcloud ftfy

import os, re, gc, random, zipfile, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from ftfy import fix_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc, roc_auc_score

import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, DataCollatorWithPadding,
    set_seed, EarlyStoppingCallback
)
from wordcloud import WordCloud

# ---------------------------
# 1) CONFIG
# ---------------------------
SEED = 42
set_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

# ✅ Usar CSV subido (si existe) y fallback a GitHub
LOCAL_CSV = "/mnt/data/data_extended (1).csv"
GITHUB_RAW_URL = "https://raw.githubusercontent.com/danielalva2008/TFM/master/TFM_UNIR/DATASETS/2.-Dataset%20Trabajados/data_extended.csv"

# ✅ ZIP solo tablas+figuras
ZIP_NAME = "TFM_SOLO_TABLAS_Y_FIGURAS.zip"
BASE_OUTPUT_DIR = "/content/outputs_tfm_solo_tablas_figuras"

# ✅ IMPORTANTE: carpeta Figuras/ como en tu LaTeX
FIG_DIR = os.path.join(BASE_OUTPUT_DIR, "Figuras")
TAB_DIR = os.path.join(BASE_OUTPUT_DIR, "tables")
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(TAB_DIR, exist_ok=True)

# ✅ Checkpoints (fuera del ZIP) -> y se borran al final de cada entrenamiento
CKPT_ROOT = "/content/_tfm_checkpoints_tmp"   # NO está dentro de BASE_OUTPUT_DIR
os.makedirs(CKPT_ROOT, exist_ok=True)

MODELS = {
    "DistilBERT-es": "dccuchile/distilbert-base-spanish-uncased",
    "ALBETO_Tiny": "dccuchile/albert-tiny-spanish",
    "MiniLM_Multilingual": "microsoft/Multilingual-MiniLM-L12-H384",
}

TEXT_COL  = "full_text"
LABEL_COL = "label"
LABEL_NAME = {0: "negativo", 1: "neutro", 2: "positivo"}
VALID_LABELS = set(LABEL_NAME.keys())

MAX_LEN = 128
TEST_SIZE = 0.15
VAL_SIZE  = 0.15

# ✅ Batch efectivo como pediste
BATCH_SIZE = 16
TARGET_EFFECTIVE_BATCH = 32
GRAD_ACC = max(1, TARGET_EFFECTIVE_BATCH // BATCH_SIZE)
print(f"✅ BATCH_SIZE={BATCH_SIZE} | TARGET_EFFECTIVE_BATCH={TARGET_EFFECTIVE_BATCH} | GRAD_ACC={GRAD_ACC}")

EPOCHS = 4
LR = 2e-5
USE_FP16 = torch.cuda.is_available()
EARLY_STOPPING_PATIENCE = 2

# ✅ SOLO columna entrenamiento (con tildes)
TEXT_FOR_TRAIN = "text_tr"

# ✅ Filtro para TF-IDF/Chi2
TFIDF_CHI2_BANNED = {"marca", "numero", "número", "escribe"}

SCENARIOS = [
    {"name": "desbalanceado", "balanced": False},
    {"name": "balanceado", "balanced": True},
]

# ---------------------------
# 2) HELPERS
# ---------------------------
def clear_gpu():
    try:
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
    except Exception:
        pass

def save_fig_jpg(path_no_ext: str):
    path = path_no_ext if path_no_ext.lower().endswith(".jpg") else (path_no_ext + ".jpg")
    plt.tight_layout()
    plt.savefig(path, dpi=220, bbox_inches="tight", format="jpg")
    plt.close()
    return path

def fix_text_utf8(s: str) -> str:
    return fix_text(str(s))

def remove_accents(text: str) -> str:
    pairs = [('á','a'), ('é','e'), ('í','i'), ('ó','o'), ('ú','u'), ('ü','u')]
    t = str(text)
    for a, b in pairs:
        t = t.replace(a, b).replace(a.upper(), b.upper())
    return t

def term_allowed_for_tfidf(term: str) -> bool:
    parts = term.split()
    return not any(p in TFIDF_CHI2_BANNED for p in parts)

def save_markdown_table(df: pd.DataFrame, md_path: str, title: str):
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(f"# {title}\n\n")
        f.write(df.to_markdown(index=False))

# ✅ NUEVO: gráficos de distribución de clases (3 gráficos pedidos)
def plot_class_distribution(df_in: pd.DataFrame, title: str, fname_noext: str):
    counts = df_in[LABEL_COL].value_counts().reindex([0,1,2]).fillna(0).astype(int)
    plt.figure(figsize=(6,4))
    plt.bar([LABEL_NAME[i] for i in [0,1,2]], counts.values)
    plt.title(title)
    plt.xlabel("Clase")
    plt.ylabel("N° de registros")
    return save_fig_jpg(os.path.join(FIG_DIR, fname_noext))

def plot_class_distribution_compare(df_des: pd.DataFrame, df_bal: pd.DataFrame, title: str, fname_noext: str):
    c1 = df_des[LABEL_COL].value_counts().reindex([0,1,2]).fillna(0).astype(int)
    c2 = df_bal[LABEL_COL].value_counts().reindex([0,1,2]).fillna(0).astype(int)

    labels = [LABEL_NAME[i] for i in [0,1,2]]
    x = np.arange(len(labels))
    width = 0.35

    plt.figure(figsize=(7,4))
    plt.bar(x - width/2, c1.values, width, label="Desbalanceado")
    plt.bar(x + width/2, c2.values, width, label="Balanceado")
    plt.xticks(x, labels)
    plt.title(title)
    plt.xlabel("Clase")
    plt.ylabel("N° de registros")
    plt.legend()
    return save_fig_jpg(os.path.join(FIG_DIR, fname_noext))

# ---------------------------
# 3) SLANG + STOPWORDS (EDA)
# ---------------------------
SLANG = [
    (r"\bmrd\b", "mierda"),
    (r"\bd\b", "de"),
    (r"\b[qk]\b", "que"),
    (r"\bxo\b", "pero"),
    (r"\bxa\b", "para"),
    (r"\b[xp]q\b", "porque"),
    (r"\bes[qk]\b", "es que"),
    (r"\bfvr\b", "favor"),
    (r"\b(xfa|xf|pf|plis|pls|porfa)\b", "por favor"),
    (r"\bdnd\b", "donde"),
    (r"\btb\b", "también"),
    (r"\b(tq|tk)\b", "te quiero"),
    (r"\b(tqm|tkm)\b", "te quiero mucho"),
    (r"\bx\b", "por"),
    (r"\+", "mas"),
]

def apply_slang(t: str) -> str:
    for pat, rep in SLANG:
        t = re.sub(pat, rep, t)
    return t

stop_words = set([
    'a','adelante','además','ahi','ahora','ahí','al','algo','alguna','algunas','alguno','algunos','algún',
    'alrededor','ambos','ante','anterior','antes','apenas','aproximadamente','aquel','aquellas','aquellos',
    'aqui','aquí','arriba','asi','así','atras','aunque','aún','ayer','b','bajo','bastante','bien','buen',
    'buena','buenas','bueno','buenos','cada','casi','cerca','cierta','ciertas','cierto','ciertos','cinco',
    'como','con','contra','cosas','creo','cual','cuales','cualquier','cuando','cuanto','cuatro','cuenta',
    'cómo','da','dado','dan','dar','de','debe','deben','debido','decir','del','demás','dentro','desde',
    'después','dia','dice','dicen','dicho','dieron','diferente','diferentes','dijeron','dijo','dio','donde',
    'dos','durante','e','ejemplo','el','ella','ellas','ello','ellos','embargo','en','encima','entonces',
    'entre','era','eran','eras','eres','es','esa','esas','ese','eso','esos','esta','estaba','estaban',
    'estado','estamos','estan','estar','estará','estas','este','esto','estos','estoy','estuvo','ex','existe',
    'existen','fin','fue','fuera','fueron','fui','fuimos','gran','grandes','ha','haber','había','habían',
    'habrá','hace','hacemos','hacen','hacer','hacia','haciendo','hago','han','has','hasta','hay','haya','he',
    'hecho','hemos','hicieron','hizo','hoy','hubo','igual','incluso','intenta','intentamos','intentan',
    'intentar','intento','ir','junto','la','lado','largo','las','le','les','llegó','lleva','llevar','lo','los',
    'luego','lugar','manera','mas','mayor','me','mediante','mejor','menos','mi','mientras','mis','misma','mismas',
    'mismo','mismos','modo','momento','mucha','muchas','mucho','muchos','muy','más','nada','nadie','ni','ninguna',
    'ningunas','ninguno','ningunos','ningún','nos','nosotras','nosotros','nuestra','nuestras','nuestro','nuestros',
    'nueva','nuevas','nuevo','nuevos','nunca','o','ocho','otra','otras','otro','otros','para','parece','parte',
    'pasada','pasado','pero','poca','pocas','poco','pocos','podemos','poder','podra','podrán','podria','podrian',
    'podrias','podría','podrían','por','porque','posible','primer','primera','primero','primeros','principalmente',
    'propia','propias','propio','propios','próximo','próximos','pudo','pueda','puede','pueden','puedo','pues','que',
    'quien','quienes','quiere','qué','quién','respecto','se','sea','sean','segunda','segundo','según','seis','sera',
    'será','serán','sería','si','sido','siempre','siendo','siete','sigue','siguiente','sin','sino','sobre','sola',
    'solamente','solas','solo','solos','somos','son','soy','su','sus','sí','también','tampoco','tan','tanto','te',
    'tendrá','tendrán','tenemos','tener','tenga','tengo','tenía','tenido','tercera','ti','tiempo','tiene','tienen',
    'toda','todas','todavía','todo','todos','total','tras','trata','través','tres','tu','tus','tuvo','tuyo','un',
    'una','unas','uno','unos','usa','usamos','usan','usar','usas','uso','usted','va','vamos','van','varias','varios',
    'vaya','veces','ven','ver','verdad','vez','vosotras','vosotros','voy','y','ya','yo',
    'á','ésta','éste','éstas','éstos','última','últimas','ultimo','último','últimos'
])

# ---------------------------
# 4) Normalización
# ---------------------------
def normalize_for_transformer(text: str) -> str:
    # Suave (con tildes/ñ/ü) para entrenamiento
    t = fix_text_utf8(text).lower()
    t = re.sub(r"http\S+|www\.\S+", " ", t)
    t = re.sub(r"@\w+", " ", t)
    t = re.sub(r"#\w+", " ", t)
    t = apply_slang(t)
    t = re.sub(r"[^a-záéíóúñü\s]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

def eda_aggressive(text: str) -> str:
    # Agresivo (sin tildes, stopwords fuera) SOLO para EDA
    t = fix_text_utf8(text).lower()
    t = apply_slang(t)
    t = remove_accents(t)
    t = re.sub(r"http\S+|www\.\S+", " ", t)
    t = re.sub(r"@\w+", " ", t)
    t = re.sub(r"#\w+", " ", t)
    t = re.sub(r"[^a-zñ\s]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    toks = [w for w in t.split() if (len(w) > 2 and w not in stop_words)]
    return " ".join(toks)

# ---------------------------
# 5) TOP 20 GENERAL (EDA normalizado: text_eda)
# ---------------------------
def tfidf_top_general(df_in: pd.DataFrame, text_col="text_eda", top_n=20, ngram_range=(1,2), min_df=3):
    texts = df_in[text_col].astype(str).tolist()
    vec = TfidfVectorizer(lowercase=True, ngram_range=ngram_range, min_df=min_df)
    X = vec.fit_transform(texts)
    terms = np.array(vec.get_feature_names_out())

    mean_tfidf = np.asarray(X.mean(axis=0)).ravel()
    order = np.argsort(mean_tfidf)[::-1]

    picked = []
    for j in order:
        term = terms[j]
        if term_allowed_for_tfidf(term):
            picked.append((term, float(mean_tfidf[j])))
        if len(picked) >= top_n:
            break
    return picked

# ---------------------------
# 6) Splits + HF datasets
# ---------------------------
def split_stratified(df_in: pd.DataFrame, label_col: str, test_size=0.15, val_size=0.15, seed=42):
    df_train, df_temp = train_test_split(
        df_in, test_size=(test_size + val_size),
        stratify=df_in[label_col], random_state=seed
    )
    rel_test = test_size / (test_size + val_size)
    df_val, df_test = train_test_split(
        df_temp, test_size=rel_test,
        stratify=df_temp[label_col], random_state=seed
    )
    return df_train.reset_index(drop=True), df_val.reset_index(drop=True), df_test.reset_index(drop=True)

def make_balanced_undersampling(df_in: pd.DataFrame, label_col: str, seed=42) -> pd.DataFrame:
    counts = df_in[label_col].value_counts()
    n_min = counts.min()
    parts = []
    for lab in sorted(counts.index):
        parts.append(df_in[df_in[label_col] == lab].sample(n=n_min, random_state=seed))
    return pd.concat(parts).sample(frac=1, random_state=seed).reset_index(drop=True)

def tokenize_dataset(ds: DatasetDict, tokenizer, text_col: str):
    def tok(batch):
        return tokenizer(batch[text_col], truncation=True, max_length=MAX_LEN)
    return ds.map(tok, batched=True)

# ---------------------------
# 7) Métricas (HF Evaluate)
# ---------------------------
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_metric.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"],
        "precision_macro": precision_metric.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall_macro": recall_metric.compute(predictions=preds, references=labels, average="macro")["recall"],
    }

# ---------------------------
# 8) Figuras: Loss + ROC + CM
# ---------------------------
def plot_loss_curves(trainer, title, fname_noext):
    hist = trainer.state.log_history
    train_steps, train_loss = [], []
    eval_steps, eval_loss = [], []

    for row in hist:
        if "loss" in row and "eval_loss" not in row and "step" in row:
            train_steps.append(row["step"])
            train_loss.append(row["loss"])
        if "eval_loss" in row and "step" in row:
            eval_steps.append(row["step"])
            eval_loss.append(row["eval_loss"])

    if len(train_steps)==0 and len(eval_steps)==0:
        return None

    plt.figure(figsize=(7,4))
    if len(train_steps)>0:
        plt.plot(train_steps, train_loss, label="train_loss")
    if len(eval_steps)>0:
        plt.plot(eval_steps, eval_loss, label="eval_loss")
    plt.title(title)
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.legend()
    return save_fig_jpg(os.path.join(FIG_DIR, fname_noext))

def plot_confusion_matrix_blue(cm, title, fname_noext):
    plt.figure(figsize=(6,5))
    plt.imshow(cm, interpolation="nearest", cmap="Blues")
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(3)
    plt.xticks(tick_marks, [LABEL_NAME[i] for i in [0,1,2]])
    plt.yticks(tick_marks, [LABEL_NAME[i] for i in [0,1,2]])
    thresh = cm.max()/2.0 if cm.max()>0 else 0.5
    for i in range(3):
        for j in range(3):
            plt.text(j, i, int(cm[i,j]),
                     ha="center", va="center",
                     color="white" if cm[i,j] > thresh else "black",
                     fontsize=12, fontweight="bold")
    plt.ylabel("Real")
    plt.xlabel("Predicción")
    return save_fig_jpg(os.path.join(FIG_DIR, fname_noext))

def plot_multiclass_roc(y_true, y_proba, title, fname_noext):
    classes = [0,1,2]
    y_bin = label_binarize(y_true, classes=classes)
    plt.figure(figsize=(7,6))
    for i, lab in enumerate(classes):
        fpr, tpr, _ = roc_curve(y_bin[:, i], y_proba[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{LABEL_NAME[lab]} (AUC={roc_auc:.3f})")
    fpr_micro, tpr_micro, _ = roc_curve(y_bin.ravel(), y_proba.ravel())
    auc_micro = auc(fpr_micro, tpr_micro)
    plt.plot(fpr_micro, tpr_micro, linestyle="--", label=f"micro-avg (AUC={auc_micro:.3f})")
    plt.plot([0,1],[0,1], linestyle=":", linewidth=1)
    plt.xlim([0.0,1.0]); plt.ylim([0.0,1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(title)
    plt.legend(loc="lower right")
    return save_fig_jpg(os.path.join(FIG_DIR, fname_noext))

# ---------------------------
# 9) Carga tokenizer/modelo (ALBETO preferir use_fast=False)
# ---------------------------
def load_tokenizer_and_model(model_ckpt: str):
    is_albert = "albert" in model_ckpt.lower()
    preferred_fast = (not is_albert)
    last_err = None
    for use_fast in [preferred_fast, (not preferred_fast)]:
        try:
            tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=use_fast)
            model = AutoModelForSequenceClassification.from_pretrained(
                model_ckpt,
                num_labels=3,
                id2label={i: LABEL_NAME[i] for i in [0,1,2]},
                label2id={LABEL_NAME[i]: i for i in [0,1,2]},
            )
            return tokenizer, model, use_fast, "OK"
        except Exception as e:
            last_err = e
    return None, None, None, f"FAIL: {last_err}"

# ---------------------------
# 10) Entrenar "como antes" + NUEVO: loss en resultados
# ---------------------------
def train_and_eval_keep_behavior(scenario_name: str, model_name_key: str, model_ckpt: str, ds: DatasetDict):
    clear_gpu()

    tokenizer, model, used_fast, status = load_tokenizer_and_model(model_ckpt)
    if tokenizer is None:
        return {
            "scenario": scenario_name, "model": model_name_key, "status": status,
            "accuracy": np.nan, "f1_macro": np.nan, "precision_macro": np.nan, "recall_macro": np.nan,
            "auc_macro_ovr": np.nan, "test_loss": np.nan, "best_val_loss": np.nan, "best_val_f1_macro": np.nan
        }

    ds_tok = tokenize_dataset(ds, tokenizer, TEXT_FOR_TRAIN)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    run_dir = os.path.join(CKPT_ROOT, f"{scenario_name}_{model_name_key}")
    if os.path.exists(run_dir):
        shutil.rmtree(run_dir, ignore_errors=True)
    os.makedirs(run_dir, exist_ok=True)

    args = TrainingArguments(
        output_dir=run_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        greater_is_better=True,
        learning_rate=LR,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACC,
        num_train_epochs=EPOCHS,
        weight_decay=0.01,
        logging_steps=50,
        report_to="none",
        seed=SEED,
        fp16=USE_FP16,
        remove_unused_columns=True
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_tok["train"],
        eval_dataset=ds_tok["validation"],
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=EARLY_STOPPING_PATIENCE)]
    )

    trainer.train()

    # ✅ curva de pérdida (train/eval)
    plot_loss_curves(
        trainer,
        f"Loss - {model_name_key} ({scenario_name})",
        f"TFM_LOSS_{scenario_name}_{model_name_key}"
    )

    # ✅ extraer best_val_loss y best_val_f1_macro desde log_history (validación)
    hist = trainer.state.log_history
    eval_losses = [h["eval_loss"] for h in hist if "eval_loss" in h]
    eval_f1s = [h.get("eval_f1_macro") for h in hist if "eval_f1_macro" in h]

    best_val_loss = float(np.min(eval_losses)) if len(eval_losses) else np.nan
    best_val_f1 = float(np.max([x for x in eval_f1s if x is not None])) if len(eval_f1s) else np.nan

    # Test
    test_metrics = trainer.evaluate(ds_tok["test"])
    preds_out = trainer.predict(ds_tok["test"])

    y_true = preds_out.label_ids
    y_pred = np.argmax(preds_out.predictions, axis=-1)

    # CM
    cm = confusion_matrix(y_true, y_pred, labels=[0,1,2])
    plot_confusion_matrix_blue(
        cm,
        f"Matriz de confusión - {model_name_key} ({scenario_name})",
        f"TFM_CM_{scenario_name}_{model_name_key}"
    )

    # ROC
    logits = preds_out.predictions
    probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()
    plot_multiclass_roc(
        y_true, probs,
        f"ROC - {model_name_key} ({scenario_name})",
        f"TFM_ROC_{scenario_name}_{model_name_key}"
    )

    # AUC macro OVR
    try:
        y_bin = label_binarize(y_true, classes=[0,1,2])
        auc_macro = roc_auc_score(y_bin, probs, average="macro", multi_class="ovr")
    except Exception:
        auc_macro = np.nan

    # ✅ BORRAR checkpoints
    shutil.rmtree(run_dir, ignore_errors=True)

    return {
        "scenario": scenario_name,
        "model": model_name_key,
        "accuracy": float(test_metrics.get("eval_accuracy", np.nan)),
        "f1_macro": float(test_metrics.get("eval_f1_macro", np.nan)),
        "precision_macro": float(test_metrics.get("eval_precision_macro", np.nan)),
        "recall_macro": float(test_metrics.get("eval_recall_macro", np.nan)),
        "auc_macro_ovr": float(auc_macro),
        "test_loss": float(test_metrics.get("eval_loss", np.nan)),   # ✅ NUEVO
        "best_val_loss": best_val_loss,                               # ✅ NUEVO
        "best_val_f1_macro": best_val_f1,                             # ✅ NUEVO
        "tokenizer_use_fast": bool(used_fast),
        "status": "OK"
    }

# ============================================================
# A) CARGAR DATASET
# ============================================================
if os.path.exists(LOCAL_CSV):
    print(f"📥 Leyendo CSV local: {LOCAL_CSV}")
    df_base = pd.read_csv(LOCAL_CSV)
else:
    print("📥 CSV local no encontrado. Leyendo GitHub RAW...")
    df_base = pd.read_csv(GITHUB_RAW_URL)

assert TEXT_COL in df_base.columns, f"Falta columna: {TEXT_COL}"
assert LABEL_COL in df_base.columns, f"Falta columna: {LABEL_COL}"

df_base = df_base[[TEXT_COL, LABEL_COL]].copy()
df_base[TEXT_COL] = df_base[TEXT_COL].fillna("").astype(str).str.strip()
df_base = df_base[df_base[TEXT_COL].str.len() > 0]

df_base[LABEL_COL] = pd.to_numeric(df_base[LABEL_COL], errors="coerce")
df_base = df_base.dropna(subset=[LABEL_COL])
df_base[LABEL_COL] = df_base[LABEL_COL].astype(int)
df_base = df_base[df_base[LABEL_COL].isin(VALID_LABELS)].reset_index(drop=True)

df_base = df_base.drop_duplicates(subset=[TEXT_COL]).reset_index(drop=True)

print(f"\n✅ Total registros válidos (base): {len(df_base)}")

# Columnas
df_base["text_tr"] = df_base[TEXT_COL].apply(normalize_for_transformer)
df_base["text_eda"] = df_base[TEXT_COL].apply(eda_aggressive)
df_base["label_name"] = df_base[LABEL_COL].map(LABEL_NAME)

# ============================================================
# B0) Construir ambos datasets (para los 3 gráficos de distribución)
# ============================================================
df_des = df_base.copy()
df_bal = make_balanced_undersampling(df_base, LABEL_COL, seed=SEED)

# ✅ 1) Distribución balanceado
plot_class_distribution(
    df_bal,
    "Distribución de clases - dataset balanceado",
    "TFM_02_Distribucion_Clases_balanceado"
)

# ✅ 2) Distribución desbalanceado
plot_class_distribution(
    df_des,
    "Distribución de clases - dataset desbalanceado",
    "TFM_01_Distribucion_Clases_desbalanceado"
)

# ✅ 3) Comparativo (nombre EXACTO como tu Cap. 4)
plot_class_distribution_compare(
    df_des, df_bal,
    "Distribución de clases en el dataset desbalanceado y balanceado",
    "TFM_03_Comp_Distribucion_Desbalanceado_vs_Balanceado"
)

# ============================================================
# B) Ejecutar escenarios: TOP 20 + WordCloud + conteos + entrenamiento
# ============================================================
all_metrics = []
train_counts_rows = []

for sc in SCENARIOS:
    scenario_name = sc["name"]
    is_balanced = sc["balanced"]

    df_sc = df_bal.copy() if is_balanced else df_des.copy()

    # Splits
    train_df, val_df, test_df = split_stratified(df_sc, LABEL_COL, test_size=TEST_SIZE, val_size=VAL_SIZE, seed=SEED)

    # ✅ Conteo de registros usados
    train_counts_rows.append({
        "scenario": scenario_name,
        "balanced": is_balanced,
        "total_escenario": len(df_sc),
        "train_usado_entrenamiento": len(train_df),
        "validation": len(val_df),
        "test": len(test_df),
    })

    print("\n" + "="*80)
    print(f"ESCENARIO: {scenario_name.upper()} | balanced={is_balanced}")
    print(f"Total escenario: {len(df_sc)} | Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")
    print("="*80)

    # ✅ WordCloud general (EDA) - nombres como tu TFM
    joined = " ".join(df_sc["text_eda"].astype(str).tolist()).strip()
    if joined:
        wc = WordCloud(width=1200, height=600, background_color="white").generate(joined)
        plt.figure(figsize=(12,6))
        plt.imshow(wc, interpolation="bilinear")
        plt.axis("off")
        plt.title(f"WordCloud general (EDA) - {scenario_name}")
        save_fig_jpg(os.path.join(FIG_DIR, f"TFM_04_WordCloud_General_{scenario_name}"))

    # ✅ TOP 20 general TF-IDF (EDA)
    top_general = tfidf_top_general(df_sc, text_col="text_eda", top_n=20)
    df_top_general = pd.DataFrame(top_general, columns=["termino", "tfidf_mean"])

    topgen_csv = os.path.join(TAB_DIR, f"TFM_TFIDF_Top20_General_{scenario_name}.csv")
    topgen_md  = os.path.join(TAB_DIR, f"TFM_TFIDF_Top20_General_{scenario_name}.md")
    df_top_general.to_csv(topgen_csv, index=False, encoding="utf-8")
    save_markdown_table(df_top_general, topgen_md, f"TFM - TF-IDF Top 20 General (EDA) - {scenario_name}")

    plt.figure(figsize=(8,4))
    plt.bar(df_top_general["termino"], df_top_general["tfidf_mean"])
    plt.title(f"TF-IDF Top 20 General (EDA) - {scenario_name}")
    plt.xticks(rotation=75, ha="right")
    save_fig_jpg(os.path.join(FIG_DIR, f"TFM_08_TFIDF_Top20_General_{scenario_name}"))

    # Entrenamiento
    ds = DatasetDict(
        train=Dataset.from_pandas(train_df[[LABEL_COL, TEXT_FOR_TRAIN]].reset_index(drop=True)),
        validation=Dataset.from_pandas(val_df[[LABEL_COL, TEXT_FOR_TRAIN]].reset_index(drop=True)),
        test=Dataset.from_pandas(test_df[[LABEL_COL, TEXT_FOR_TRAIN]].reset_index(drop=True)),
    )

    for model_key, ckpt in MODELS.items():
        res = train_and_eval_keep_behavior(scenario_name, model_key, ckpt, ds)
        all_metrics.append(res)

# ============================================================
# C) Guardar tabla de conteos (CSV + MD)
# ============================================================
df_counts = pd.DataFrame(train_counts_rows)
counts_csv = os.path.join(TAB_DIR, "TFM_Conteo_Registros_Entrenamiento_Escenarios.csv")
counts_md  = os.path.join(TAB_DIR, "TFM_Conteo_Registros_Entrenamiento_Escenarios.md")
df_counts.to_csv(counts_csv, index=False, encoding="utf-8")
save_markdown_table(df_counts, counts_md, "TFM - Registros usados en entrenamiento por escenario")

# ============================================================
# D) Guardar tabla final de métricas + LOSS (CSV + MD)
# ============================================================
df_metrics = pd.DataFrame(all_metrics)
df_metrics = df_metrics[
    ["scenario","model",
     "accuracy","f1_macro","precision_macro","recall_macro","auc_macro_ovr",
     "test_loss","best_val_loss","best_val_f1_macro",   # ✅ NUEVO
     "tokenizer_use_fast","status"]
].sort_values(by=["scenario","f1_macro"], ascending=[True, False])

metrics_csv = os.path.join(TAB_DIR, "TFM_Metricas_Comparativa_Modelos.csv")
metrics_md  = os.path.join(TAB_DIR, "TFM_Metricas_Comparativa_Modelos.md")
df_metrics.to_csv(metrics_csv, index=False, encoding="utf-8")
save_markdown_table(df_metrics, metrics_md, "TFM - Métricas comparativas (modelos x escenarios) + LOSS")

print("\n✅ Tablas guardadas en:", TAB_DIR)
print("✅ Figuras guardadas en:", FIG_DIR)

# ============================================================
# E) ZIP SOLO con tablas y Figuras (NO checkpoints)
# ============================================================
zip_path = os.path.join("/content", ZIP_NAME)

with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for folder in [FIG_DIR, TAB_DIR]:
        for root, dirs, files in os.walk(folder):
            for file in files:
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, BASE_OUTPUT_DIR)
                z.write(full_path, arcname=os.path.join("outputs_tfm_solo_tablas_figuras", rel_path))

print(f"\n✅ ZIP (solo tablas+figuras) listo: {zip_path}")

# limpieza final de checkpoints
shutil.rmtree(CKPT_ROOT, ignore_errors=True)
os.makedirs(CKPT_ROOT, exist_ok=True)

# Descarga automática en Colab
from google.colab import files
files.download(zip_path)
